In [32]:
#library(gclus)
library(car)

library(dplyr)

ERROR: Error in library(gclus): there is no package called ‘gclus’


In [7]:
trainset <- read.csv("../Downloads/Predict_churn/train.csv")

In [8]:
members <- read.csv("../Downloads/Predict_churn/members.csv")

In [9]:
sample_submission <- read.csv("../Downloads/Predict_churn/sample_submission_zero.csv")

In [10]:
transactions <- read.csv("../Downloads/Predict_churn/transactions.csv")

In [12]:
trainset[1:5,]

msno,is_churn
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1


In [13]:
members[1:5,]

msno,city,bd,gender,registered_via,registration_init_time,expiration_date
URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,,9,20150525,20150526
U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,,4,20161221,20161224
W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,,4,20160306,20160309
1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,20161031,20161107
SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,,4,20170202,20170205


In [14]:
sample_submission[1:5,]

msno,is_churn
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0


In [15]:
transactions[1:5,]
# For each user, there are several payment methods, 
# payment plans, plan list price, actual amount paid, auto renewals
# and transaction dates

msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [21]:
train_transactions <- trainset %>% 
    left_join(transactions, by = "msno")

Warning message:
“Column `msno` joining factors with different levels, coercing to character vector”

In [24]:
length(train_transactions$msno)

[1] 15883148

In [25]:
length(transactions$msno)

[1] 21547746

[1] 2363626

In [ ]:
length(unique(transactions$msno))

In [20]:
length(trainset$msno)

[1] 992931

In [26]:
train_transactions[1:5,]

msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,7,0,0,0,20161031,20161107,0
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,30,149,149,0,20170107,20170206,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20161031,20161221,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20160715,20160821,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20160831,20161021,0


In [27]:
# Scatterplot Matrices from the glus Package
library(gclus)
dta <- train_transactions[c(2:10)] # get data
dta.r <- abs(cor(dta)) # get correlations
dta.col <- dmat.color(dta.r) # get colors
# reorder variables so those with highest correlation
# are closest to the diagonal
dta.o <- order.single(dta.r)
cpairs(dta, dta.o, panel.colors=dta.col, gap=.5,
main="Variables Ordered and Colored by Correlation" ) 

ERROR: Error in library(gclus): there is no package called ‘gclus’


In [ ]:
scatterplotMatrix(~payment_plan_days+plan_list_price|is_churn, data=train_transactions[1:100,],
   main="Three Cylinder Options")

In [ ]:
attach(mtcars)
plot(wt, mpg, main="Scatterplot Example",
   xlab="Car Weight ", ylab="Miles Per Gallon ", pch=19) 